# Mandatory imports and utils

In [ ]:
{-# LANGUAGE BangPatterns, ScopedTypeVariables #-}
import Control.Monad
import Control.Monad.Primitive
import Control.Parallel.Strategies
import Control.Concurrent
import Control.Exception (evaluate)

import qualified Data.Set as Set
import qualified Data.Vector.Unboxed as U

import Numeric.SpecFunctions
import Numeric.MathFunctions.Constants
import Numeric.MathFunctions.Comparison
import Numeric.Polynomial
import Numeric.Polynomial.Chebyshev

import Text.Printf(printf)

import IHaskell.Display
import Graphics.Rendering.Chart.Backend.Cairo
import Graphics.Rendering.Chart.Easy hiding (within)

import Debug.Trace

:l NB/Plot
:l NB/Heatmap

greyColormap 0.2
_heat_map_values (def :: HeatMap Double Double Double)

setNumCapabilities 6

# Incomplete beta

Quick reminder about beta function and (regularized) incomplete beta functions:

Beta function:
$$B(a,b) = \int_0^1 t^{a-1}(1 - t)^{b-1} \,dt $$

Incomplete beta:
$$B(x; a,b) = \int_0^x t^{a-1}(1 - t)^{b-1} \,dt \qquad x \in [0,1]$$

Regularized incomplete beta (from now it'll be referred to simply as incomplete beta)
$$I(x; a,b) = \frac{B(x; a,b)}{B(a,b)}$$


## Debugging of [math-functions#35](https://github.com/bos/math-functions/issues/35)

Originally uncovered when plotting roundtrip error of `cumulative . quantile` for beta distribution in log scale.

In [ ]:
let fun x = let p  = invIncompleteBeta 7 0.07 x
                x' = incompleteBeta    7 0.07 p
            in x'
toRenderable
  $ plotFunctionsLog [\x -> logBase 10 $ relativeError (fun x) x] (1e-10, 1)

Oh! It's looks bad let plot how does incomplete beta looks like.

In [ ]:
-- Bird's eye view
toRenderable
  $ plotFunctions [invIncompleteBeta 7 0.07] (1e-7,1e-6)
-- Closeup
toRenderable
  $ plotFunctions [invIncompleteBeta 7 0.07] (2.8e-7,3e-7)

It's strange dip in smooth and _monotonic_ function! After adding couple of `traceShow`'s to incompleteBeta implementation it becomes clear that problem is lack of convergence. Initial guess is bad and 10 iterations is not enough. So we need to modify `invIncompleteBeta` for easy of debugging. We add  pluggable oracle for initial guess and return number of iterations together with result.

We'll need to make quite a few modifications. We'll need to fix math-functions#36 so we'll have to add custom `logBeta` definition.

In [ ]:
logGammaLL :: Double -> Double
logGammaLL x
    | x <= 0    = m_pos_inf
    | x <  1    = lanczos (1+x) - log x
    
    | x >= 1 && x < 2 =  lanczos (1+x) - log x
    | otherwise = lanczos x
    where
      -- Evaluate Lanczos approximation for γ=6
      lanczos z = fini 
                $ U.foldl' go (L 0 (z+7)) a
        where
          fini (L l _)   = log (l+a0) + log m_sqrt_2_pi - z65 + (z-0.5) * log z65
          go   (L l t) k = L (l + k / t) (t-1)
          z65 = z + 6.5
      a0  = 0.9999999999995183
      a   = U.fromList [ 0.1659470187408462e-06
                       , 0.9934937113930748e-05
                       , -0.1385710331296526
                       , 12.50734324009056
                       , -176.6150291498386
                       , 771.3234287757674
                       , -1259.139216722289
                       , 676.5203681218835
                       ]
data L = L {-# UNPACK #-} !Double {-# UNPACK #-} !Double

-- | Compute the natural logarithm of the beta function.
logBeta' :: Double -> Double -> Double
logBeta' a b
    | p < 0     = m_NaN
    | p == 0    = m_pos_inf
    | p >= 10   = log q * (-0.5) + m_ln_sqrt_2_pi + logGammaCorrection p + c +
                  (p - 0.5) * log ppq + q * log1p(-ppq)
    | q >= 10   = logGamma p + c + p - p * log pq + (q - 0.5) * log1p(-ppq)
    | otherwise = logGammaLL p + logGammaLL q - logGammaLL pq
    where
      p   = min a b
      q   = max a b
      ppq = p / pq
      pq  = p + q
      c   = logGammaCorrection q - logGammaCorrection pq
      -- Fix another issue in current Lanczos approximation 
      logGammaLL z | z < 1 = logGammaL (z+1) - log z
                   | otherwise = logGammaL z

-- | Compute the log gamma correction factor for @x@ &#8805; 10.  This
-- correction factor is suitable for an alternate (but less
-- numerically accurate) definition of 'logGamma':
--
-- >lgg x = 0.5 * log(2*pi) + (x-0.5) * log x - x + logGammaCorrection x
logGammaCorrection :: Double -> Double
logGammaCorrection x
    | x < 10    = m_NaN
    | x < big   = chebyshevBroucke (t * t * 2 - 1) coeffs / x
    | otherwise = 1 / (x * 12)
  where
    big    = 94906265.62425156
    t      = 10 / x
    coeffs = U.fromList [
               0.1666389480451863247205729650822e+0,
              -0.1384948176067563840732986059135e-4,
               0.9810825646924729426157171547487e-8,
              -0.1809129475572494194263306266719e-10,
               0.6221098041892605227126015543416e-13,
              -0.3399615005417721944303330599666e-15,
               0.2683181998482698748957538846666e-17
             ]

incompleteBeta'' :: Double -- ^ /p/ > 0
               -> Double -- ^ /q/ > 0
               -> Double -- ^ /x/, must lie in [0,1] range
               -> Double
incompleteBeta'' p q = incompleteBeta_ (logBeta' p q) p q

In [ ]:
-- | Regularized incomplete beta function. Uses algorithm AS63 by
-- Majumder and Bhattachrjee and quadrature approximation for large
-- /p/ and /q/.
incompleteBeta' :: Double -- ^ /p/ > 0
               -> Double -- ^ /q/ > 0
               -> Double -- ^ /x/, must lie in [0,1] range
               -> Double
incompleteBeta' p q = incompleteBeta_ b p q
  where
    b = logBeta' p q
    
    
-- | Regularized incomplete beta function. Same as 'incompleteBeta'
-- but also takes logarithm of beta function as parameter.
incompleteBeta_ :: Double -- ^ logarithm of beta function for given /p/ and /q/
                -> Double -- ^ /p/ > 0
                -> Double -- ^ /q/ > 0
                -> Double -- ^ /x/, must lie in [0,1] range
                -> Double
incompleteBeta_ beta p q x
  | p <= 0 || q <= 0            =
      error $ printf "incompleteBeta_: p <= 0 || q <= 0. p=%g q=%g x=%g" p q x
  | x <  0 || x >  1 || isNaN x =
      error $ printf "incompleteBeta_: x out of [0,1] range. p=%g q=%g x=%g" p q x
  | x == 0 || x == 1            = x
  | p >= (p+q) * x   = incompleteBetaWorker beta p q x
  | otherwise        = 1 - incompleteBetaWorker beta q p (1 - x)

-- Worker for incomplete beta function. It is separate function to
-- avoid confusion with parameter during parameter swapping
incompleteBetaWorker :: Double -> Double -> Double -> Double -> Double
incompleteBetaWorker beta p q x
  -- For very large p and q this method becomes very slow so another
  -- method is used.
  | p > 3000 && q > 3000 = error "incompleteBetaApprox beta p q x"
  | otherwise            = loop (p+q) (truncate $ q + cx * (p+q)) 1 1 1
  where
    -- Constants
    eps = m_epsilon
    cx  = 1 - x
    -- Loop
    loop !psq (ns :: Int) ai term betain
      | done      = betain' * x ** p * cx ** (q - 1) / p / exp beta
      | otherwise = loop psq' (ns - 1) (ai + 1) term' betain'
      where
        -- New values
        term'   = term * fact / (p + ai)
        betain' = betain + term'
        fact | ns >  0   = (q - ai) * x/cx
             | ns == 0   = (q - ai) * x
             | otherwise = psq * x
        -- Iterations are complete
        done = db <= eps && db <= eps*betain' where db = abs term'
        psq' = if ns < 0 then psq + 1 else psq    

And finally inverse incomplete beta

In [ ]:
-- | Data type to describe approximation used. See 
data Guess
  = AS109
  | AS64_2
  | NR1
  | NR2
  deriving (Show,Eq)

data Res = Res
  { result :: !Double
  , nIter  :: !Int
  , iters  :: [Double]
  }
  deriving Show 

-- | Compute inverse of regularized incomplete beta function. Uses
-- initial approximation from AS109, AS64 and Halley method to solve
-- equation.
invIncompleteBeta' 
  :: (Double -> Double -> Double -> (Guess,Double)) -- Initial guess
  -> (Double -> Double -> Double -> Double)         -- incomplete beta
  -> Double     -- ^ /p/ > 0
  -> Double     -- ^ /q/ > 0
  -> Double     -- ^ /a/ ∈ [0,0.5] !!!
  -> Res
invIncompleteBeta' oracle calcIbeta p q a
  | p <= 0 || q <= 0 =
      error $ printf "invIncompleteBeta p <= 0 || q <= 0.  p=%g q=%g a=%g" p q a
  | a <  0 || a >  1 =
      error $ printf "invIncompleteBeta x must be in [0,1].  p=%g q=%g a=%g" p q a
  | a == 0 || a == 1 = Res a 0 []
  | a > 0.5          = error "not implemented"
  | otherwise        = invIncompleteBetaWorker oracle calcIbeta (logBeta p q) p q  a


invIncompleteBetaWorker
  :: (Double -> Double -> Double -> (Guess,Double))
  -> (Double -> Double -> Double -> Double)
  -> Double -> Double -> Double -> Double -> Res
-- NOTE: p <= 0.5.
invIncompleteBetaWorker oracle calcIbeta beta a b p = loop (0::Int) [] (snd $ oracle a b p)
  where
    a1 = a - 1
    b1 = b - 1
    -- Solve equation using Halley method
    done i xs x = Res { result = x 
                      , nIter  = i
                      , iters  = reverse $ x : xs
                      }
    loop !i !xs !x
      -- We cannot continue at this point so we simply return `x'
      | traceShow (x,f,f',u) False = undefined      
      | isNaN x' = error $ "Got to the NaN a="++show a++" b="++show b++" p="++show p++" x_last="++show x
      
      | x == 0 || x == 1             = done i xs x
      -- When derivative becomes infinite we cannot continue
      -- iterations. It can only happen in vicinity of 0 or 1. It's
      -- hardly possible to get good answer in such circumstances but
      -- `x' is already reasonable.
      | isInfinite f'                = done i xs x
      -- Iterations limit reached. Most of the time solution will
      -- converge to answer because of discreteness of Double. But
      -- solution have good precision already.
      | i >= 100                     = traceShow (a,b,p) $ done i xs x
      -- Solution converges      
      | abs dx <= 16 * m_epsilon * x = done i xs x'
      | otherwise                    = loop (i+1) (x:xs) x'
      where
        -- Calculate Halley step.
        f   = calcIbeta a b x  - p
        f'  = exp $ a1 * log x + b1 * log1p (-x) - beta
        u   = f / f'
        dx  = u / (1 - 0.5 * min 1 (u * (a1 / x - b1 / (1 - x))))
        -- Next approximation. If Halley step leads us out of [0,1]
        -- range we revert to bisection.
        x'  | z < 0     = x / 2
            | z > 1     = (x + 1) / 2
            | otherwise = z
            where z = x - dx

-- Calculate initial guess. Approximations from AS64, AS109 and
-- Numerical recipes are used.
--
-- Equations are referred to by name of paper and number e.g. [AS64 2]
-- In AS64 papers equations are not numbered so they are refered
-- to by number of appearance starting from definition of
-- incomplete beta.
guessIIBeta :: Double -> Double -> Double -> (Guess,Double)
guessIIBeta a b p
      -- In this region we use approximation from AS109 (Carter
      -- approximation). It's reasonably good (2 iterations on
      -- average)
      | a > 1 && b > 1 =
          let r = (y*y - 3) / 6
              s = 1 / (2*a - 1)
              t = 1 / (2*b - 1)
              h = 2 / (s + t)
              w = y * sqrt(h + r) / h - (t - s) * (r + 5/6 - 2 / (3 * h))
          in (AS109, a / (a + b * exp(2 * w)))
      -- Otherwise we revert to approximation from AS64 derived from
      -- [AS64 2] when it's applicable.
      --
      -- It slightly reduces average number of iterations when `a' and
      -- `b' have different magnitudes.
      | chi2 > 0 && ratio > 1 = (AS64_2, 1 - 2 / (ratio + 1))
      -- If all else fails we use approximation from "Numerical
      -- Recipes". It's very similar to approximations [AS64 4,5] but
      -- it never goes out of [0,1] interval.
      | otherwise = case () of
          _| p < t / w  -> (NR1, (a * p * w) ** (1/a))
           | otherwise  -> (NR2, 1 - (b * (1 - p) * w) ** (1/b))
           where
             lna = log $ a / (a+b)
             lnb = log $ b / (a+b)
             t   = exp( a * lna ) / a
             u   = exp( b * lnb ) / b
             w   = t + u
      where
        -- Formula [2]
        ratio = (4*a + 2*b - 2) / chi2
        -- Quantile of chi-squared distribution. Formula [3].
        chi2 = 2 * b * (1 - t + y * sqrt t) ** 3
          where
            t   = 1 / (9 * b)
        -- `y' is Hasting's approximation of p'th quantile of standard
        -- normal distribution.
        y   = r - ( 2.30753 + 0.27061 * r )
                  / ( 1.0 + ( 0.99229 + 0.04481 * r ) * r )
          where
            r = sqrt $ - 2 * log p

incompleteBetaDeriv :: Double -> Double -> Double -> Double
incompleteBetaDeriv a b p
  = p**(a-1) * (1-p)**(b-1)

--invIncompleteBeta' guessIIBeta 7 0.07 2.8e-7
--invIncompleteBeta' guessIIBeta 7 0.07 2.88e-7

# Checking convergence

Pretty bad. Let plot number of iterations:

In [ ]:
toRenderable
  $ plotFunctions [fromIntegral . nIter . invIncompleteBeta' guessIIBeta incompleteBeta 7 0.07
                  , const 10] (1e-7,1e-6)
guessIIBeta 7 0.07 4e-7

That's pretty bad. Initial guess is very very poor. And it's initial guess from AS64 fails us. Let look just how bad situation is.

In [ ]:
nIterations =
  [ ((a1,b1),(a2,b2), fromIntegral (maximum iters) :: Double)
  | (a1,a,a2) <- linspaceIntervals (0,1) n
  , (b1,b,b2) <- linspaceIntervals (0,1) n
        -- Calculate number of iterations
  , let iters = parMap rpar (nIter . invIncompleteBeta' guessIIBeta incompleteBeta a b) $ linspace (0,0.5) 100
  ]
  where n  = 20 :: Int

void $ evaluate (nIterations `using` parList rdeepseq)

toRenderable $ 
  layout_plots .~ 
    [ toPlot $ heat_map_values .~ nIterations
             $ def
             ]
  $ def

maximum [z | (_,_,z) <- nIterations]

Situation is pretty grim. Let look just how bad it is

In [ ]:
toRenderable
  $ plotFunctionsLog [ fromIntegral . nIter . invIncompleteBeta' guessIIBeta incompleteBeta 0.5 0.5
                     , const 10
                     ] (1e-10, 0.45)
mapM_ print $ take 10 $ iters
 $ invIncompleteBeta' guessIIBeta incompleteBeta 0.5 0.5 1.743132149343274e-10

Ah! We got ourselves into infinite loop without reaching convergence! Let look at incomplete beta there

It looks like we lost precision. And math-functions\#37 is started here

# After fixing incompelte beta

In [ ]:
nIterations =
  [ ((a1,b1),(a2,b2), fromIntegral (maximum iters) :: Double)
  | (a1,a,a2) <- linspaceIntervals (0,1) n
  , (b1,b,b2) <- linspaceIntervals (0,1) n
        -- Calculate number of iterations
  , let iters = parMap rpar (nIter . invIncompleteBeta' guessIIBeta incompleteBeta' a b) $ linspace (0,0.5) 100
  ]
  where n  = 20 :: Int
void $ evaluate (nIterations `using` parList rdeepseq)

toRenderable $ 
  layout_plots .~ 
    [ toPlot $ heat_map_values .~ nIterations
             $ heat_map_color_map .~ blackbodyColormap
             $ def
             ]
  $ def

maximum [z | (_,_,z) <- nIterations]

Now we need to do something with fake nonconvergenece

In [ ]:
dropLoop xs
  = map fst
  $ takeWhile (\(x,vals) -> not $ Set.member x vals)
  $ zip xs sets
  where
    sets = scanl (flip Set.insert) Set.empty xs
    
plotIterationsMap aRng bRng n
  = layout_title .~ ("a = " ++ show aRng ++ " b = " ++ show bRng ++ " max=" ++ show (maxN))
  $ layout_plots .~ 
    [ toPlot $ heat_map_values    .~ nIterations
             $ heat_map_color_map .~ blackbodyColormap
             $ def
             ]
  $ def
  where
    maxN = maximum [z | (_,_,z) <- nIterations]
    nIterations = flip using (parList rdeepseq) $ 
      [ ((a1,b1),(a2,b2), fromIntegral (maximum iters) :: Double)
      | (a1,a,a2) <- linspaceIntervals aRng n
      , (b1,b,b2) <- linspaceIntervals bRng n
        -- Calculate number of iterations
      , let iters = map (cleverNIter . invIncompleteBeta' guessIIBeta incompleteBeta' a b)
                  $ logspace (1e-10,0.5) 100
      ]
    --
    cleverNIter :: Res -> Int
    cleverNIter r 
      | n == 100  = length $ dropLoop $ iters r
      | otherwise = n
      where
        n   = nIter r
        
plotGuessRatio aRng bRng n approx
  = layout_title .~ ("a = " ++ show aRng ++ " b = " ++ show bRng ++ " " ++ show approx)
  $ layout_plots .~ 
    [ toPlot $ heat_map_values    .~ nIterations
             $ heat_map_color_map .~ blackbodyColormap
             $ def
             ]
  $ def
  where
    nIterations = flip using (parList rdeepseq) $ 
      [ ((a1,b1),(a2,b2), length guesses)
      | (a1,a,a2) <- linspaceIntervals aRng n
      , (b1,b,b2) <- linspaceIntervals bRng n
        -- Calculate number of iterations
      , let guesses = filter (==approx) 
                    $ map (fst . guessIIBeta a b)
                    $ logspace (1e-10,0.5) 100
      ]
    --

## Let check which initial guesses are used where

### AS109 guess

Carter approximation. Used when:

$$\begin{aligned}
 a > 1 \\
 b > 1 \\
\end{aligned}$$

$$\begin{aligned}
x &= \frac{a}{a + b \exp(2w)} \\
w &= \frac{y \sqrt{h + r}}{h} 
   - (t - s)\left(r + \frac{5}{6} - \frac{2}{3h}\right) \\
r &= \frac{y^2 - 3}{6}\\
s &= \frac{1}{2a - 1} \\
t &= \frac{1}{2b - 1} \\
h &= \frac{2}{s + t}  \\
\end{aligned}$$

$y$ is Hasting's approximation of p'th quantile of standard normal distribution.

$$\begin{aligned}
 y &= r' - \frac{2.30753 + 0.27061 r'}{1.0 + 0.99229 r' + 0.04481 r'^2} \\
 r' &= \sqrt{-2\log p}
\end{aligned}$$

In [ ]:
toRenderable $ plotGuessRatio (0,20) (0,20) 40 AS109
toRenderable $ plotGuessRatio (0,20) (0,20) 40 AS64_2
toRenderable $ plotGuessRatio (0,20) (0,20) 40 NR1
toRenderable $ plotGuessRatio (0,2)  (0,2)  40 NR2

## Рассмотрим теперь как выглядят распределение худшего числа итераций

### AS109

In [ ]:
toRenderable $ plotIterationsMap (1,50)  (1,50)  40
toRenderable $ plotIterationsMap (1,50)  (2,50)  40
toRenderable $ plotIterationsMap (1,100) (1,2)   30
toRenderable $ plotIterationsMap (1,2)   (1,200) 20
toRenderable $ plotIterationsMap (1,4)   (1,400) 30

In [ ]:
toRenderable $ plotIterationsMap (10,1000) (10,1000) 40

In [ ]:
--incompleteBeta' 345.125 938.125 0.1954382572000985
--invIncompleteBeta 9 2 1e-300
incompleteBeta 9 2 7.316450683654291e-42

In [ ]:
invIncompleteBeta 345.125 938.125 1e-10 

In [ ]:
toRenderable $ plotIterationsMap (1,100) (1,400) 30

Итак, у нас есть проблемы с при a >> 1 и малым (b-1), а также при больших a

# Квадрат 1

In [ ]:
toRenderable $ plotIterationsMap (0,1) (0,1) 40

In [ ]:
toRenderable $ plotIterationsMap (0,20) (0,1.2) 40

In [ ]:
toRenderable $ plotIterationsMap (0,1) (0,1) 40

In [ ]:
toRenderable $ plotIterationsMap (0,0.05) (0,1) 40

In [ ]:
toRenderable $ plotIterationsMap (6,8) (0.05,0.10) 40

In [ ]:
temmeGuess a b p
  | a + b >= 5 = case () of
    _| sqrt minA > (maxA-minA)  && minA > 5     -> error "Sec.2"
--     | lambda > 0.2 && lambda < 0.8 && a_b > 10 -> error "Sec. 3 / or power method"
  where 
    --
    minA = min a b
    maxA = max a b
    a_b  = a + b
    
    beta = a - b
    mu   = (a - b) / a


-- | Initial guess from  section 2.
ibetaTemmeGuessSect2 :: Double -> Double -> Double -> Double
ibetaTemmeGuessSect2 a b p
  = 0 
  where
    -- Calculate even 
    beta  = b - a
    beta2 = beta  * beta
    beta3 = beta2 * beta
    eta0  = - sqrt(2 / a) * invErfc (2 * p)
    eta   = evaluatePolynomialL (1/a)
      [ eta0
        -- Eq. following 2.15
      , evaluatePolynomialL eta0
        [ -beta * m_sqrt_2 / 2
        , (1 - 2*beta) / 8
        , -(beta * m_sqrt_2) / 48
        , -1 /192
        , beta * m_sqrt_2 / 3840
        ]
        -- Eq. following 2.17
      , evaluatePolynomial eta0
        [ beta * m_sqrt_2 * (3*beta - 2) / 12
        , (20*beta2 - 12*beta + 1) / 128
        ,  beta * m_sqrt_2 * (20 * beta - 1) / 960
        , (16 * beta2 + 30 * beta - 15) / 4608
        , beta * m_sqrt_2 * (21 * beta + 32) / 53760
        , (-32 * beta2 + 63) / 368640
        , -beta * m_sqrt_2 * (120 * beta + 17) / 25804480
        ]
      , evaluatePolynomial eta0
        [ B * r2 * (-75 * B_2 + 80 * B - 16) / 480
        , (-1080 * B_3 + 868 * B_2 - 90 * B - 45) / 9216
        , B * r2 * (-1190 * B_2 + 84 * B + 373) / 53760
        , (-2240 * B_3 - 2508 * B_2 + 2100 * B - 165) / 368640
        ]
      ]
    -- 

In [ ]:
tets x
  | x > 1 | x > 2 = 0
          | otherwise = 1
  | otherwise = x

In [ ]:
m_sqrt_2

In [ ]:
:t evaluatePolynomialL